In [3]:
import math 
from math import factorial
import numpy as np

# Binomial Trees

Binomial Trees are a useful technique for pricing options and other derivatives. They represent different possible price paths that might be followed by the asset underlying the derivatives contract. The general assumption for binomial trees is that the underlying's price follows a random walk. They also incorporate the argument of no-arbitrage and the principle of risk-neutral valuation.

This function is able to calculate both European and American options using binomial trees. It also displays the whole tree structure. Any number of steps can be calculated, and results for European options should converge to the results given by the Black-Scholes model.

In [29]:
def binomial_tree(s, k, t, r, q, vol, n_steps, type_opt='european', call_put='call'):

    # Each step considers the time to maturity divided by the number of steps
    delta_t = t/n_steps

    # We calculate the upward and downward probabilities based on the given volatility
    u = math.exp(vol*math.sqrt(delta_t)) 
    d = 1/u
    a = math.exp((r-q)*delta_t)
    p = (a-d)/(u-d)

    # We build the stock's tree. It will later be merged with the option's tree
    tree = {}
    for i in np.arange(n_steps+1):
        inner_tree = {}
        for j in np.arange(start=0, stop=i+1):
            inner_tree[j] = s*(u**(i-j))*(d**(j))
        tree[i] = inner_tree

    # We create dictionaries that will be used to create the option's tree
    opt_tree = {}
    opt_tree_last = {}
    # We calculate the option's value at the last step. This values will be backpropagated to construct the option's tree
    if call_put=='call':
        for ky,v in tree[n_steps].items():
            poff = np.max([v-k,0])
            opt_tree_last[ky] = poff
    elif call_put=='put':
        for ky,v in tree[n_steps].items():
            poff = np.max([k-v,0])
            opt_tree_last[ky] = poff
    opt_tree[n_steps] = opt_tree_last   # Here, we store the option's payoff of the last step. This values will be backpropagated 

    if type_opt=='american':
        # We backpropagate the option and fill up the option's tree
        for i in list(tree.keys())[::-1][:-1]:
            dict_opt = {}
            for n in np.arange(i):
                fu = opt_tree[i][n]
                fd = opt_tree[i][n+1]
                f = math.exp(-r*delta_t)*(p*fu+(1-p)*fd)
                # Since the option is American and can be exercised before the maturity of the option, we have to check for the real value of the option at any given 
                # time. If the option is more valuable when exercising, the difference between spot and strike prices (according to whether it's a call or put option)
                # then its price will be such a difference. If not, it will be the present value of the expected value of the option for the given probabilities
                if call_put=='call':
                    diff = tree[i-1][n]-k
                elif call_put=='put':
                    diff = k-tree[i-1][n]        
                dict_opt[n] = np.max([f, diff])
            opt_tree[i-1] = dict_opt
    
    elif type_opt=='european':
        # We backpropagate the option and fill up the option's tree
        for i in list(tree.keys())[::-1][:-1]:
            dict_opt = {}
            for n in np.arange(i):
                fu = opt_tree[i][n]
                fd = opt_tree[i][n+1]
                f = math.exp(-r*delta_t)*(p*fu+(1-p)*fd)
                dict_opt[n] = f
            opt_tree[i-1] = dict_opt

    binom_tree = {}
    for ky,v in tree.items():
        level_dict = {}
        for k1, prices in v.items():
            option_price = opt_tree[ky][k1]
            level_dict[k1] = [np.round(prices,4), np.round(option_price,4)]
        binom_tree[ky] = level_dict

    return {'price':np.round(f,4), 'tree':binom_tree}

In [28]:
binomial_tree(s=31, k=30, t=9/12, r=0.05, vol=0.3, q=0.05, 
              n_steps=3, type_opt='american', call_put='put')

{'price': 2.8356,
 'tree': {0: {0: [31.0, 2.84]},
  1: {0: [36.0169, 0.93], 1: [26.6819, 4.54]},
  2: {0: [41.8456, 0.0], 1: [31.0, 1.76], 2: [22.9654, 7.03]},
  3: {0: [48.6177, 0.0],
   1: [36.0169, 0.0],
   2: [26.6819, 3.32],
   3: [19.7665, 10.23]}}}

# References
Hull, J. C. (2015). Options, Futures, and Other Derivatives (9th ed.). Toronto: Pearson Education Inc.